In [1]:
import pandas as pd
import numpy as np

In [2]:
season_dat = pd.read_csv('data/DataFiles/RegularSeasonCompactResults.csv')
season_dat.shape

(161552, 8)

In [3]:
tourney_dat = pd.read_csv('data/DataFiles/NCAATourneyCompactResults.csv')
tourney_dat.shape

(2184, 8)

In [4]:
season_dat.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,1985,20,1228,81,1328,64,N,0
1,1985,25,1106,77,1354,70,H,0
2,1985,25,1112,63,1223,56,H,0
3,1985,25,1165,70,1432,54,H,0
4,1985,25,1192,86,1447,74,H,0


## Limit data to 2004 onwards. 

In [5]:
szn_dat = season_dat[season_dat['Season']>=2004]
szn_dat.shape

(82888, 8)

In [6]:
t_dat = tourney_dat[tourney_dat['Season']>=2004]
t_dat.shape

(984, 8)

In [7]:
### Identify tourney teams

In [8]:
tourney_teams_part_a = t_dat[['Season','WTeamID']].rename(columns={'WTeamID':'TeamID'})
tourney_teams_part_b = t_dat[['Season','LTeamID']].rename(columns={'LTeamID':'TeamID'})

In [9]:
tourney_teams = pd.concat([tourney_teams_part_a,tourney_teams_part_b]).reset_index(drop=True).drop_duplicates() 

In [10]:
# confirm number of tourney teams per year
tourney_teams.groupby('Season').agg('count')

,TeamID
Season,
2004,65
2005,65
2006,65
2007,65
2008,65
2009,65
2010,65
2011,68
2012,68


In [11]:
## Extract games for only teams that were in the tourney that year.  

In [12]:
szn_tourney_teams_part_a = pd.merge(szn_dat,tourney_teams,how='inner',
                                   left_on=['Season', 'WTeamID'], 
                                   right_on=['Season', 'TeamID'])
szn_tourney_teams_part_b = pd.merge(szn_dat,tourney_teams,how='inner',
                                   left_on=['Season', 'LTeamID'], 
                                   right_on=['Season', 'TeamID'])

In [13]:
szn_tourney_teams = pd.concat([szn_tourney_teams_part_a,szn_tourney_teams_part_b])
szn_tourney_teams = szn_tourney_teams.drop('TeamID',axis=1)
szn_tourney_teams = szn_tourney_teams.fillna(0)
szn_tourney_teams.shape

(31712, 8)

In [14]:
szn_tourney_teams = szn_tourney_teams.drop_duplicates()
szn_tourney_teams.shape

(27301, 8)

In [65]:
def w_year_df(row):
    year_ext_df = tourney_teams[tourney_teams['Season'] == row['Season']]
    return (row['WTeamID'] in year_ext_df['TeamID'].tolist())

In [66]:
def l_year_df(row):
    year_ext_df = tourney_teams[tourney_teams['Season'] == row['Season']]
    return (row['LTeamID'] in year_ext_df['TeamID'].tolist())

In [67]:
szn_tourney_teams['WTID_in_tourney'] = szn_tourney_teams.apply(w_year_df,axis=1)

In [68]:
szn_tourney_teams['LTID_in_tourney'] = szn_tourney_teams.apply(l_year_df,axis=1)

In [77]:
szn_tourney_teams.to_csv('data/Final/network_adj_szn.csv',index=False)